In [1]:
from keras import applications
import matplotlib.pyplot as plt
import os
import cv2

%matplotlib inline


Using TensorFlow backend.


In [2]:
model = applications.VGG16(weights='imagenet', include_top=True)

In [3]:
model.layers[-1].get_config()

{'activation': 'softmax',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'predictions',
 'trainable': True,
 'units': 1000,
 'use_bias': True}

In [4]:
model.layers[-2].get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'fc2',
 'trainable': True,
 'units': 4096,
 'use_bias': True}

In [5]:
from keras.models import Model

penultimate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('fc2').output)

In [6]:
BASE_DATA_DIR = '/home/ubuntu/Downloads/examples/'
total_im_path_list = []
for album_name in os.listdir(BASE_DATA_DIR):
    cur_im_list = os.listdir(os.path.join(BASE_DATA_DIR, album_name))
    total_im_path_list.extend([os.path.join(BASE_DATA_DIR,album_name,im_name) for im_name in cur_im_list])

In [7]:
total_vec_list = []
for im_ind, im_file_path in enumerate(total_im_path_list):
    if im_ind % 10 ==0:
        print (im_ind)
    img = cv2.imread(im_file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(224,224))
    feature_vec = penultimate_layer_model.predict(img[None])
    total_vec_list.append(feature_vec.ravel())


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160


In [8]:
import numpy as np

In [9]:
total_vec_mat = np.array(total_vec_list)

In [10]:
total_vec_mat.shape

(162, 4096)

In [11]:
from sklearn.neighbors import NearestNeighbors

In [12]:
neigh = NearestNeighbors(n_neighbors=2)

In [13]:
neigh.fit(total_vec_mat,range(total_vec_mat.shape[0]))

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

In [25]:
neigh.kneighbors(total_vec_mat[10])

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([[  0.        ,  85.99409784]]), array([[10, 11]]))

In [26]:
total_im_path_list[10]

'/home/ubuntu/Downloads/examples/album2/53.jpg'

In [24]:
total_im_path_list[27]

'/home/ubuntu/Downloads/examples/album2/46.jpg'

In [20]:
len(total_im_path_list)

162